In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, element
import time

In [2]:
base_url = "https://www.pro-football-reference.com/players/"
list_of_players = ["Kyler Murray", "Daniel Jones", "Dwayne Haskins", "Drew Lock", "Will Grier", "Ryan Finley", "Jarrett Stidham", "Easton Stick", "Clayton Thorson", "Gardener Minshew II", "Trace McSorely"]
pick_NO = [1,6,15,42,100,104,133,166,167,178,197]
list_of_years = ["2019", "2020", "2021", "2022", "2023"]
player_urls = ["M/MurrKy00/", "J/JoneDa05/", "H/HaskDw00/", "L/LockDr00/", "G/GrieWi00/", "F/FinlRy00/", "S/StidJa00/", "S/SticEa00/", "T/ThorCl00/", "M/MinsGa00/", "M/McSoTr00/"]

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

In [5]:
stats= []
for player, pick, player_Name in zip(player_urls, pick_NO, list_of_players):
    for year in list_of_years:
        url = f"{base_url}{player}/gamelog/{year}"
        api_response = requests.get(url, headers=headers)
        time.sleep(3)
        soup = BeautifulSoup(api_response.text, 'html.parser')
        table = soup.find("tbody")
        if table:
            for row in table.find_all("tr"):
                game_row = []
                if not row.find("td", class_="center"):
                    game_row.append(player_Name)
                    game_row.append(pick)
                    
                    # Game date
                    try:
                        game_row.append(row.find("td", {"data-stat": "game_date"}).text)
                    except (AttributeError, ValueError):
                        game_row.append("")

                    # Game result
                    try:
                        game_row.append(row.find("td", {"data-stat": "game_result"}).text[0])
                    except (AttributeError, IndexError):
                        game_row.append("")

                    # Passing completions
                    try:
                        pass_cmp = row.find("td", {"data-stat": "pass_cmp"}).text
                        game_row.append(int(pass_cmp) if pass_cmp else 0)
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing attempts
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_att"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing yards
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_yds"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passing touchdowns
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_td"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Interceptions
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_int"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Passer rating
                    try:
                        game_row.append(float(row.find("td", {"data-stat": "pass_rating"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0.0)

                    # Times sacked
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "pass_sacked"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing attempts
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_att"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing yards
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_yds"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Rushing touchdowns
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "rush_td"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    # Fumbles
                    try:
                        game_row.append(int(row.find("td", {"data-stat": "fumbles"}).text))
                    except (AttributeError, ValueError):
                        game_row.append(0)

                    stats.append(game_row)

In [6]:
df = pd.DataFrame(stats, columns=[
    'Player', 'Pick', 'Game_Date', 'Result', 'Pass_Cmp', 
    'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Pass_Rating', 
    'Pass_Sacked', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Fumbles'
])
df

,Player,Pick,Game_Date,Result,Pass_Cmp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Pass_Rating,Pass_Sacked,Rush_Att,Rush_Yds,Rush_TD,Fumbles
0,Kyler Murray,1,2019-09-08,T,29,54,308,2,1,75.2,5,3,13,0,0
1,Kyler Murray,1,2019-09-15,L,25,40,349,0,0,90.5,3,3,4,0,1
2,Kyler Murray,1,2019-09-22,L,30,43,173,2,2,73.1,8,8,69,0,0
3,Kyler Murray,1,2019-09-29,L,22,32,241,0,1,77.7,4,4,27,1,0
4,Kyler Murray,1,2019-10-06,W,20,32,253,0,0,87.1,1,10,93,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Trace McSorely,197,2022-11-13,W,0,0,0,0,0,0.0,0,0,0,0,0
255,Trace McSorely,197,2022-11-21,L,6,10,59,0,1,37.1,0,1,6,0,0
256,Trace McSorely,197,2022-12-18,L,7,15,95,0,2,27.8,1,1,8,0,0
257,Trace McSorely,197,2022-12-25,L,24,45,217,0,1,57.4,1,7,14,0,2
